## Filepath

In [1]:
# Andrew

filename = input()

#C:/Users/AMCBR/MyStuff/College Notes & Work/Year 4/Machine Learning & Data Mining/Assignment 3/owls15.csv

C:/Users/AMCBR/MyStuff/College Notes & Work/Year 4/Machine Learning & Data Mining/Assignment 3/owls15.csv


## Imports

In [3]:
# Catherine

import pandas as pd
import numpy as np
from numpy import array, dot, random
import random
from pandas import read_csv

np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

import pandas as pd
import math
import itertools

# Functions

## Catherine

In [4]:
# Catherine
### Data Prep

def scale_and_encode(df):

    # Get Features
    # if features are continuous - revisit
    feats = df.drop(df.columns[[-1]], axis=1).apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

    # Get encoded Labels
    labels = pd.get_dummies(df[df.columns[-1:]])
    

    # Recombine feats & labels
    combine_data = pd.concat([feats, labels], axis=1)
    
    return combine_data, len(feats.columns), list(labels.columns.values)


# Gets data in the correct format and adds a bias to the features
def feat_label_bias(df, label_ind):

    list_data_pre = df.values.tolist()
    list_data = [[x[:label_ind] + [1.0] for x in list_data_pre]] + [[x[label_ind:] for x in list_data_pre]]
    
    new_df = pd.DataFrame(data=list_data).transpose()
    new_df.columns = ['feats', 'labels']

    
    return new_df

In [15]:
# Catherine
# Perceptron Funcs

def sigmoid(x):
    return 1/(1+math.e**-x)

def update_weights(w, learning_rate, error, feature_vec):
    return w + [(learning_rate * error) * f for f in feature_vec] 

def train_perceptron(train_dat):

    
    num_feats = len(train_dat.head(1).values[0][0]) 
    num_classes = len(train_dat.head(1).values[0][1]) 

    
    w = np.random.rand(num_classes, num_feats) 


  
    errors = [] 
    learning_rate = 0.12
    curr_iter = 0
    iterations = 300
    num_epochs = 2
    
    for epoch in range(0, num_epochs):
    
        for row in train_dat.iterrows():

            # Use specified number of training data points
            if curr_iter < iterations:
                curr_iter = curr_iter + 1

                # Feats and labels
                tp_feats = row[1][0]
                tp_labels = row[1][1]
                class_pred = [0.0] * len(tp_labels)

                # Calculate class prediction list 
                preds_list = []

                for i in range(0, num_classes):

                    pred = sigmoid(dot(w[i], tp_feats)) # weights by features
                    preds_list.append(pred)


                class_pred[np.argmax(preds_list)] = 1.0

                for i in range(0, num_classes):

                    error = tp_labels[i] - preds_list[i]
                    errors.append(error)

                    w[i] = update_weights(w[i], learning_rate, error, tp_feats)


    return w

In [6]:
# Catherine
# Predictions & Testing perceptron

def predict_one_case(w, feature_list):

    num_classes, num_feats = w.shape
    
    
    # Calculate class prediction list 
    preds_list = []
    class_pred = [0.0] * num_classes

    for i in range(0, num_classes):

        pred = sigmoid(dot(w[i], feature_list)) # weights by features
        preds_list.append(pred)

    class_pred[np.argmax(preds_list)] = 1.0

    return class_pred



def predict(w, pred_data):
    
    new_df_list = []
    
    for row in pred_data.iterrows():
        feats = row[1][0]
        
        curr_pred = predict_one_case(w, feats)
        
        new_df_row = [feats, curr_pred]
        new_df_list.append(new_df_row)
    
    new_df = pd.DataFrame(data=new_df_list)
    new_df.columns = ['feats', 'prediction']
    
    return new_df


def test_perceptron_accuracy(w, test_data):
    
    new_df_list = []
    
    for row in test_data.iterrows():
        feats = row[1][0]
        labels = row[1][1]
        
        curr_pred = predict_one_case(w, feats)
        
        new_df_row = [labels, curr_pred]
        new_df_list.append(new_df_row)
    
    new_df = pd.DataFrame(data=new_df_list)
    new_df.columns = ['labels', 'prediction']
    
    num_true = 0
    num_false = 0
    
    for row in new_df.iterrows():
        labs = row[1][0]
        preds = row[1][1]
        
        if labs == preds:
            num_true = num_true + 1
        else:
            num_false = num_false + 1
    
    accuracy = num_true/(num_true+num_false)
#     print('T: ', num_true)
#     print('F: ', num_false)
    
    return accuracy, new_df

## Andrew

In [16]:
def load_file(filename, names):
    """ Loads a csv into a dataframe"""
    
    print(filename)
    data = read_csv(filename, names=names) 
    return data


def train_test_split(data, train_frac=0.666):
    """ Takes a dataset and splits it into a training and test set with measures to ensure even distribution
    
    Keyword Arguments:
    data -- pandas dataframe of data to be split
    train_frac -- franction of the data to be used for training, the rest will be used for test
    
    N.B: The feature labes must be the last column in the dataset"""
    
    labels = data[data.columns[-1]] # get last column in dataframe which will contain labels
    num_labels = len(labels.unique())
    split_threshold = 1/num_labels
    
    train = data.sample(frac=train_frac, random_state=np.random.RandomState())
    test = data.drop(train.index)

    unique_labels = labels.unique()
  
    label_splits_train = [sum(train[train.columns[-1]] == x) for x in unique_labels]
    label_splits_test = [sum(test[test.columns[-1]] == x) for x in unique_labels]   
    
    split_vals_train = [x/len(train) for x in label_splits_train]
    split_vals_test = [x/len(test) for x in label_splits_test]
    
    is_even_split_train = [(split_val < (split_threshold + 0.1)) & (split_val > (split_threshold - 0.1))
                           for split_val in split_vals_train] # Ensure a fair split 
    is_even_split_test = [(split_val < (split_threshold + 0.1)) & (split_val > (split_threshold - 0.1))
                           for split_val in split_vals_test]
    
    if not is_even_split_train and not is_even_split_test:
        return train_test_split(data, train_frac)
    else:
        return train, test
    
    #split_val: decimal representation of how much one type makes up of the while data set
    #split_threshold: how much each of the labels should represent in the data set
    
    
def rand_train_test_split(data, train_frac=0.666):
    """ Takes a dataset and splits it into a training and test set
    
    Keyword Arguments:
    data -- pandas dataframe of data to be split
    train_frac -- franction of the data to be used for training, the rest will be used for test
    
    N.B: The feature labes must be the last column in the dataset"""
    
    labels = data[data.columns[-1]] # get last column in dataframe which will contain labels
    
    sample_size = int(len(data)*train_frac)
    
    train = data.sample(frac=train_frac, random_state=np.random.RandomState())
    test = data.drop(train.index)
    
    return train, test


def n_random_split(n, data, splitter=rand_train_test_split):
    """ Takes a pandas dataframe and returns n amount of random splits of training and test data in a dict
    
    Keyword Arguments:
    n -- number of training/test sets to return
    data -- dataframe to split 
    splitter -- function which splits the data into train and test"""
    
    d = {}
    for i in range(n):
        train, test = splitter(data)
        d[i] = [train, test]
        
    return d
    

# MAIN - single train/test cycle

In [17]:
#Andrew

# Load Data
in_data = pd.read_csv(filename, header=None, sep=',', index_col=False)

# Scale the features & Encode the labels
data_scaled, label_ind, data_labels = scale_and_encode(in_data)

# Get data in the right form & add bias
data_with_bias = feat_label_bias(data_scaled, label_ind)

# Split Data into Training and Test Sets
train_data, test_data = rand_train_test_split(data_with_bias, train_frac=0.666)

# Train perceptron
perceptron_weights = train_perceptron(train_data)

# Check the accuracy of the perceptron
accuracy, prediction_data = test_perceptron_accuracy(perceptron_weights, test_data)

print('Accuracy = ', accuracy)

Accuracy =  0.7333333333333333


## Avg. Over 10 Iterations

In [9]:
#Andrew

d = n_random_split(10, data_with_bias)

train_sets=[]
test_sets=[]
for index, datasets in d.items():
    train_sets.append(datasets[0])
    test_sets.append(datasets[1])

accuracy_scores=[] 
scores_by_iteration = {}
    
for i in range(len(train_sets)):
    perceptron_weights = train_perceptron(train_sets[i])
    accuracy, prediction_data = test_perceptron_accuracy(perceptron_weights, test_sets[i])
    accuracy_scores.append(accuracy)
    scores_by_iteration[i] = [accuracy, prediction_data]
    
print(accuracy_scores)
avg = sum(accuracy_scores)/len(accuracy_scores)
print("\nAverage Accuracy over 10 iterations: {}".format(avg))

[0.6222222222222222, 0.8, 0.8666666666666667, 0.8222222222222222, 0.7333333333333333, 0.6888888888888889, 0.8444444444444444, 0.6888888888888889, 0.8888888888888888, 0.9111111111111111]

Average Accuracy over 10 iterations: 0.7866666666666667


##  Output to file

In [10]:
# Andrew

file = open('output.txt', 'w')


file.write('The 3 columns correspond to:\n {}\n\n'.format(str(data_labels)))

for key, val in scores_by_iteration.items():
    file.write("Iteration {}:\n\n".format(key))
    file.write(str(prediction_data))
    file.write('\nAccuracy: {}'.format(val[0]))
    file.write("\n\n")


file.write("\nAverage Accuracy over 10 iterations: {}".format(avg))    #if data is a pandas.DataFrame object
file.close()

# -------------------------------------------------------------------------------------------------------

# Tests

In [14]:
# Andrew

passed = False

####Test: train_test_split
print('Test: train_test_split')
frac=0.666
train, test = train_test_split(data1, train_frac=frac)
correct_size = (len(train)+len(test) == size)
correct_split = (frac+0.1 > len(train)/size > frac-0.1 ) 

passed = correct_size and correct_split
print('Passed: {}'.format(passed))



####Test: rand_train_test_split
print('\nTest: rand_train_test_split')
frac=0.666
train, test = rand_train_test_split(data1, train_frac=frac)
correct_size = (len(train)+len(test) == size)
correct_split = (frac+0.1 > len(train)/size > frac-0.1 ) 

passed = correct_size and correct_split
print('Passed: {}'.format(passed))

Test: train_test_split
Passed: True

Test: rand_train_test_split
Passed: True
